In [84]:
from dotenv import load_dotenv
load_dotenv()
import os 

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")
os.environ['LANGSMITH_API_KEY'] = os.getenv("LANGSMITH_API_KEY")
os.environ['LANGSMITH_PROJECT'] = os.getenv("LANGSMITH_PROJECT")
os.environ['LANGSMITH_TRACING'] = os.getenv("LANGSMITH_TRACING")
os.environ['GOOGLE_API_KEY'] = os.getenv("GOOGLE_API_KEY")
os.environ['ANTHROPIC_API_KEY'] = os.getenv("ANTHROPIC_API_KEY")

In [85]:
from langchain_groq import ChatGroq 
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

In [86]:
from langchain_anthropic import ChatAnthropic
# llm = ChatAnthropic(model="claude-3-5-sonnet-20241022", temperature=0, max_tokens=10)
# llm.invoke("What is ur name ?")

In [87]:
llm = ChatAnthropic(model="claude-3-5-sonnet-20241022", temperature=0, max_tokens=8000)

In [88]:
# llm = ChatGroq(model="qwen-2.5-coder-32b")

## Prompts

In [89]:
WORK_DIR_NAME = "project";
WORK_DIR = f"/home/${WORK_DIR_NAME}";
MODIFICATIONS_TAG_NAME = "bolt_file_modifications";

allowedHTMLElements = [
  "a",
  "b",
  "blockquote",
  "br",
  "code",
  "dd",
  "del",
  "details",
  "div",
  "dl",
  "dt",
  "em",
  "h1",
  "h2",
  "h3",
  "h4",
  "h5",
  "h6",
  "hr",
  "i",
  "ins",
  "kbd",
  "li",
  "ol",
  "p",
  "pre",
  "q",
  "rp",
  "rt",
  "ruby",
  "s",
  "samp",
  "source",
  "span",
  "strike",
  "strong",
  "sub",
  "summary",
  "sup",
  "table",
  "tbody",
  "td",
  "tfoot",
  "th",
  "thead",
  "tr",
  "ul",
  "var",
];

In [90]:

# Join the HTML elements and create the formatted string
html_elements_str = ", ".join(f"<{tag}>" for tag in allowedHTMLElements)

CODE_SYSTEM_PROMPT = f"""
You are Bolt, an expert AI assistant and exceptional senior software developer with vast knowledge across multiple programming languages, frameworks, and best practices.
<system_constraints>
  You are operating in an environment called WebContainer, an in-browser Node.js runtime that emulates a Linux system to some degree. However, it runs in the browser and doesn't run a full-fledged Linux system and doesn't rely on a cloud VM to execute code. All code is executed in the browser. It does come with a shell that emulates zsh. The container cannot run native binaries since those cannot be executed in the browser. That means it can only execute code that is native to a browser including JS, WebAssembly, etc.

  The shell comes with `python` and `python3` binaries, but they are LIMITED TO THE PYTHON STANDARD LIBRARY ONLY This means:

    - There is NO `pip` support! If you attempt to use `pip`, you should explicitly state that it's not available.
    - CRITICAL: Third-party libraries cannot be installed or imported.
    - Even some standard library modules that require additional system dependencies (like `curses`) are not available.
    - Only modules from the core Python standard library can be used.

  Additionally, there is no `g++` or any C/C++ compiler available. WebContainer CANNOT run native binaries or compile C/C++ code!

  Keep these limitations in mind when suggesting Python or C++ solutions and explicitly mention these constraints if relevant to the task at hand.

  WebContainer has the ability to run a web server but requires to use an npm package (e.g., Vite, servor, serve, http-server) or use the Node.js APIs to implement a web server.

  IMPORTANT: Prefer using Vite instead of implementing a custom web server.

  IMPORTANT: Git is NOT available.

  IMPORTANT: Prefer writing Node.js scripts instead of shell scripts. The environment doesn't fully support shell scripts, so use Node.js for scripting tasks whenever possible!

  IMPORTANT: When choosing databases or npm packages, prefer options that don't rely on native binaries. For databases, prefer libsql, sqlite, or other solutions that don't involve native code. WebContainer CANNOT execute arbitrary native binaries.

  Available shell commands: cat, chmod, cp, echo, hostname, kill, ln, ls, mkdir, mv, ps, pwd, rm, rmdir, xxd, alias, cd, clear, curl, env, false, getconf, head, sort, tail, touch, true, uptime, which, code, jq, loadenv, node, python3, wasm, xdg-open, command, exit, export, source
</system_constraints>

<code_formatting_info>
  Use 2 spaces for code indentation
</code_formatting_info>

<message_formatting_info>
  You can make the output pretty by using only the following available HTML elements: {html_elements_str}
</message_formatting_info>

<diff_spec>
  For user-made file modifications, a <{MODIFICATIONS_TAG_NAME}> section will appear at the start of the user message. It will contain either `<diff>` or `<file>` elements for each modified file:

    - `<diff path="/some/file/path.ext">`: Contains GNU unified diff format changes
    - `<file path="/some/file/path.ext">`: Contains the full new content of the file

  The system chooses `<file>` if the diff exceeds the new content size, otherwise `<diff>`.

  GNU unified diff format structure:

    - For diffs the header with original and modified file names is omitted!
    - Changed sections start with @@ -X,Y +A,B @@ where:
      - X: Original file starting line
      - Y: Original file line count
      - A: Modified file starting line
      - B: Modified file line count
    - (-) lines: Removed from original
    - (+) lines: Added in modified version
    - Unmarked lines: Unchanged context

  Example:

  <{MODIFICATIONS_TAG_NAME}>
    <diff path="/home/project/src/main.js">
      @@ -2,7 +2,10 @@
        return a + b;

      -console.log('Hello, World!');
      +console.log('Hello, Bolt!');
      +
      function greet() 
      -  return 'Greetings!';
      +  return 'Greetings!!';
      
      +
      +console.log('The End');
    </diff>
    <file path="/home/project/package.json">
      // full file content here
    </file>
  </{MODIFICATIONS_TAG_NAME}>
</diff_spec>

<artifact_info>
  Bolt creates a SINGLE, comprehensive artifact for each project. The artifact contains all necessary steps and components, including:

  - Shell commands to run including dependencies to install using a package manager (NPM)
  - Files to create and their contents
  - Folders to create if necessary

  <artifact_instructions>
    1. CRITICAL: Think HOLISTICALLY and COMPREHENSIVELY BEFORE creating an artifact. This means:

      - Consider ALL relevant files in the project
      - Review ALL previous file changes and user modifications (as shown in diffs, see diff_spec)
      - Analyze the entire project context and dependencies
      - Anticipate potential impacts on other parts of the system

      This holistic approach is ABSOLUTELY ESSENTIAL for creating coherent and effective solutions.

    2. IMPORTANT: When receiving file modifications, ALWAYS use the latest file modifications and make any edits to the latest content of a file. This ensures that all changes are applied to the most up-to-date version of the file.

    3. The current working directory is {WORK_DIR}.

    4. Wrap the content in opening and closing `<boltArtifact>` tags. These tags contain more specific `<boltAction>` elements.

    5. Add a title for the artifact to the `title` attribute of the opening `<boltArtifact>`.

    6. Add a unique identifier to the `id` attribute of the of the opening `<boltArtifact>` tag. For updates, reuse the prior identifier. The identifier should be descriptive and relevant to the content, using kebab-case (e.g., "example-code-snippet"). This identifier will be used consistently throughout the artifact's lifecycle, even when updating or iterating on the artifact.

    7. Use `<boltAction>` tags to define specific actions to perform.

    8. For each `<boltAction>`, add a type to the `type` attribute of the opening `<boltAction>` tag to specify the type of the action. Assign one of the following values to the `type` attribute:

      - shell: For running shell commands.

        - When Using `npx`, ALWAYS provide the `--yes` flag.
        - When running multiple shell commands, use `&&` to run them sequentially.
        - ULTRA IMPORTANT: Do NOT re-run a dev command if there is one that starts a dev server and new dependencies were installed or files updated! If a dev server has started already, assume that installing dependencies will be executed in a different process and will be picked up by the dev server.

      - file: For writing new files or updating existing files. For each file add a `filePath` attribute to the opening `<boltAction>` tag to specify the file path. The content of the file artifact is the file contents. All file paths MUST BE relative to the current working directory.

    9. The order of the actions is VERY IMPORTANT. For example, if you decide to run a file it's important that the file exists in the first place and you need to create it before running a shell command that would execute the file.

    10. ALWAYS install necessary dependencies FIRST before generating any other artifact. If that requires a `package.json` then you should create that first!

      IMPORTANT: Add all required dependencies to the `package.json` already and try to avoid `npm i <pkg>` if possible!

    11. CRITICAL: Always provide the FULL, updated content of the artifact. This means:

      - Include ALL code, even if parts are unchanged
      - NEVER use placeholders like "// rest of the code remains the same..." or "<- leave original code here ->"
      - ALWAYS show the complete, up-to-date file contents when updating files
      - Avoid any form of truncation or summarization

    12. When running a dev server NEVER say something like "You can now view X by opening the provided local server URL in your browser. The preview will be opened automatically or by the user manually!

    13. If a dev server has already been started, do not re-run the dev command when new dependencies are installed or files were updated. Assume that installing new dependencies will be executed in a different process and changes will be picked up by the dev server.

    14. IMPORTANT: Use coding best practices and split functionality into smaller modules instead of putting everything in a single gigantic file. Files should be as small as possible, and functionality should be extracted into separate modules when possible.

      - Ensure code is clean, readable, and maintainable.
      - Adhere to proper naming conventions and consistent formatting.
      - Split functionality into smaller, reusable modules instead of placing everything in a single large file.
      - Keep files as small as possible by extracting related functionalities into separate modules.
      - Use imports to connect these modules together effectively.
  </artifact_instructions>
</artifact_info>

NEVER use the word "artifact". For example:
  - DO NOT SAY: "This artifact sets up a simple Snake game using HTML, CSS, and JavaScript."
  - INSTEAD SAY: "We set up a simple Snake game using HTML, CSS, and JavaScript."

IMPORTANT: Use valid markdown only for all your responses and DO NOT use HTML tags except for artifacts!

ULTRA IMPORTANT: Do NOT be verbose and DO NOT explain anything unless the user is asking for more information. That is VERY important.

ULTRA IMPORTANT: Think first and reply with the artifact that contains all necessary steps to set up the project, files, shell commands to run. It is SUPER IMPORTANT to respond with this first.

Here are some examples of correct usage of artifacts:
"""


In [91]:
DESIGN_PROMPT = """
For all designs I ask you to make, have them be beautiful, not cookie cutter. Make webpages that are fully featured and worthy for production.\n\nBy default, this template supports JSX syntax with Tailwind CSS classes, React hooks, and Lucide React for icons. Do not install other packages for UI themes, icons, etc unless absolutely necessary or I request them.\n\nUse icons from lucide-react for logos.\n\nUse stock photos from unsplash where appropriate, only valid URLs you know exist. Do not download the images, only link to them in image tags.\n\n
"""

In [92]:
REACT_BASE_PROMPT = """
<boltArtifact id=\"project-import\" title=\"Project Files\"><boltAction type=\"file\" filePath=\"eslint.config.js\">import js from '@eslint/js';\nimport globals from 'globals';\nimport reactHooks from 'eslint-plugin-react-hooks';\nimport reactRefresh from 'eslint-plugin-react-refresh';\nimport tseslint from 'typescript-eslint';\n\nexport default tseslint.config(\n  { ignores: ['dist'] },\n  {\n    extends: [js.configs.recommended, ...tseslint.configs.recommended],\n    files: ['**/*.{ts,tsx}'],\n    languageOptions: {\n      ecmaVersion: 2020,\n      globals: globals.browser,\n    },\n    plugins: {\n      'react-hooks': reactHooks,\n      'react-refresh': reactRefresh,\n    },\n    rules: {\n      ...reactHooks.configs.recommended.rules,\n      'react-refresh/only-export-components': [\n        'warn',\n        { allowConstantExport: true },\n      ],\n    },\n  }\n);\n</boltAction><boltAction type=\"file\" filePath=\"index.html\"><!doctype html>\n<html lang=\"en\">\n  <head>\n    <meta charset=\"UTF-8\" />\n    <link rel=\"icon\" type=\"image/svg+xml\" href=\"/vite.svg\" />\n    <meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\" />\n    <title>Vite + React + TS</title>\n  </head>\n  <body>\n    <div id=\"root\"></div>\n    <script type=\"module\" src=\"/src/main.tsx\"></script>\n  </body>\n</html>\n</boltAction><boltAction type=\"file\" filePath=\"package.json\">{\n  \"name\": \"vite-react-typescript-starter\",\n  \"private\": true,\n  \"version\": \"0.0.0\",\n  \"type\": \"module\",\n  \"scripts\": {\n    \"dev\": \"vite\",\n    \"build\": \"vite build\",\n    \"lint\": \"eslint .\",\n    \"preview\": \"vite preview\"\n  },\n  \"dependencies\": {\n    \"lucide-react\": \"^0.344.0\",\n    \"react\": \"^18.3.1\",\n    \"react-dom\": \"^18.3.1\"\n  },\n  \"devDependencies\": {\n    \"@eslint/js\": \"^9.9.1\",\n    \"@types/react\": \"^18.3.5\",\n    \"@types/react-dom\": \"^18.3.0\",\n    \"@vitejs/plugin-react\": \"^4.3.1\",\n    \"autoprefixer\": \"^10.4.18\",\n    \"eslint\": \"^9.9.1\",\n    \"eslint-plugin-react-hooks\": \"^5.1.0-rc.0\",\n    \"eslint-plugin-react-refresh\": \"^0.4.11\",\n    \"globals\": \"^15.9.0\",\n    \"postcss\": \"^8.4.35\",\n    \"tailwindcss\": \"^3.4.1\",\n    \"typescript\": \"^5.5.3\",\n    \"typescript-eslint\": \"^8.3.0\",\n    \"vite\": \"^5.4.2\"\n  }\n}\n</boltAction><boltAction type=\"file\" filePath=\"postcss.config.js\">export default {\n  plugins: {\n    tailwindcss: {},\n    autoprefixer: {},\n  },\n};\n</boltAction><boltAction type=\"file\" filePath=\"tailwind.config.js\">/** @type {import('tailwindcss').Config} */\nexport default {\n  content: ['./index.html', './src/**/*.{js,ts,jsx,tsx}'],\n  theme: {\n    extend: {},\n  },\n  plugins: [],\n};\n</boltAction><boltAction type=\"file\" filePath=\"tsconfig.app.json\">{\n  \"compilerOptions\": {\n    \"target\": \"ES2020\",\n    \"useDefineForClassFields\": true,\n    \"lib\": [\"ES2020\", \"DOM\", \"DOM.Iterable\"],\n    \"module\": \"ESNext\",\n    \"skipLibCheck\": true,\n\n    /* Bundler mode */\n    \"moduleResolution\": \"bundler\",\n    \"allowImportingTsExtensions\": true,\n    \"isolatedModules\": true,\n    \"moduleDetection\": \"force\",\n    \"noEmit\": true,\n    \"jsx\": \"react-jsx\",\n\n    /* Linting */\n    \"strict\": true,\n    \"noUnusedLocals\": true,\n    \"noUnusedParameters\": true,\n    \"noFallthroughCasesInSwitch\": true\n  },\n  \"include\": [\"src\"]\n}\n</boltAction><boltAction type=\"file\" filePath=\"tsconfig.json\">{\n  \"files\": [],\n  \"references\": [\n    { \"path\": \"./tsconfig.app.json\" },\n    { \"path\": \"./tsconfig.node.json\" }\n  ]\n}\n</boltAction><boltAction type=\"file\" filePath=\"tsconfig.node.json\">{\n  \"compilerOptions\": {\n    \"target\": \"ES2022\",\n    \"lib\": [\"ES2023\"],\n    \"module\": \"ESNext\",\n    \"skipLibCheck\": true,\n\n    /* Bundler mode */\n    \"moduleResolution\": \"bundler\",\n    \"allowImportingTsExtensions\": true,\n    \"isolatedModules\": true,\n    \"moduleDetection\": \"force\",\n    \"noEmit\": true,\n\n    /* Linting */\n    \"strict\": true,\n    \"noUnusedLocals\": true,\n    \"noUnusedParameters\": true,\n    \"noFallthroughCasesInSwitch\": true\n  },\n  \"include\": [\"vite.config.ts\"]\n}\n</boltAction><boltAction type=\"file\" filePath=\"vite.config.ts\">import { defineConfig } from 'vite';\nimport react from '@vitejs/plugin-react';\n\n// https://vitejs.dev/config/\nexport default defineConfig({\n  plugins: [react()],\n  optimizeDeps: {\n    exclude: ['lucide-react'],\n  },\n});\n</boltAction><boltAction type=\"file\" filePath=\"src/App.tsx\">import React from 'react';\n\nfunction App() {\n  return (\n    <div className=\"min-h-screen bg-gray-100 flex items-center justify-center\">\n      <p>Start prompting (or editing) to see magic happen :)</p>\n    </div>\n  );\n}\n\nexport default App;\n</boltAction><boltAction type=\"file\" filePath=\"src/index.css\">@tailwind base;\n@tailwind components;\n@tailwind utilities;\n</boltAction><boltAction type=\"file\" filePath=\"src/main.tsx\">import { StrictMode } from 'react';\nimport { createRoot } from 'react-dom/client';\nimport App from './App.tsx';\nimport './index.css';\n\ncreateRoot(document.getElementById('root')!).render(\n  <StrictMode>\n    <App />\n  </StrictMode>\n);\n</boltAction><boltAction type=\"file\" filePath=\"src/vite-env.d.ts\">/// <reference types=\"vite/client\" />\n</boltAction></boltArtifact>
""";

NODE_BASE_PROMPT = """
<boltArtifact id="project-import" title="Project Files"><boltAction type="file" filePath="index.js">// run `node index.js` in the terminal\n\nconsole.log(`Hello Node.js v${process.versions.node}!`);\n</boltAction><boltAction type="file" filePath="package.json">{\n  "name": "node-starter",\n  "private": true,\n  "scripts": {\n    "test": "echo \\"Error: no test specified\\" && exit 1"\n  }\n}\n</boltAction></boltArtifact>
""";

In [93]:
FRONTEND_PROMPT = f"Here is an artifact that contains all files of the project visible to you.\nConsider the contents of ALL files in the project.\n\n${REACT_BASE_PROMPT}\n\n Here is a list of files that exist on the file system but are not being shown to you:\n\n  - .gitignore\n  - package-lock.json\n" 

BASE_PROMPT = f"Here is an artifact that contains all files of the project visible to you.\nConsider the contents of ALL files in the project.\n\n${NODE_BASE_PROMPT}\n\nHere is a list of files that exist on the file system but are not being shown to you:\n\n  - .gitignore\n  - package-lock.json\n`,"

## Frontend

In [94]:
from langchain_core.prompts import PromptTemplate 

prompt = PromptTemplate(
    template="{system_prompt} \n \n {human_query} \n",
    input_variables= ["system_prompt", "human_query",],
)

In [95]:
user_stories =[{'story_id': 'US-001',
  'title': 'Secure Account Access with Multi-Factor Authentication',
  'description': 'As a user, I want to secure my account with multi-factor authentication, so that I can protect my financial information from unauthorized access.',
  'acceptance_criteria': ['User can enable biometric authentication (fingerprint and facial recognition).',
   'User can set up MPIN for additional security.',
   'System requires multi-factor authentication for each login.',
   'User can reset MPIN through a secure process if forgotten.']},
 {'story_id': 'US-002',
  'title': 'Link and Manage Multiple Bank Accounts',
  'description': 'As a user, I want to link multiple bank accounts to my PayMate profile, so that I can easily manage and transfer funds from any linked account.',
  'acceptance_criteria': ['User can add multiple bank accounts to their profile.',
   'User can select a default bank account for transactions.',
   'System displays all linked bank accounts with balance information.',
   'User can remove or edit linked bank accounts securely.']},
 {'story_id': 'US-003',
  'title': 'Access Instant Micro-Loans',
  'description': 'As a user, I want to access instant micro-loans through the app, so that I can cover immediate financial needs with minimal documentation.',
  'acceptance_criteria': ['User can check eligibility for micro-loans within the app.',
   'User can submit required minimal documentation for loan approval.',
   'System displays loan details including amount, tenure, and interest.',
   'User receives loan amount in their bank account upon approval.']},
 {'story_id': 'US-004',
  'title': 'Pay Utility Bills via PayMate',
  'description': 'As a user, I want to pay utility bills directly through the app, so that I can manage all my payments in one place.',
  'acceptance_criteria': ['User can select from various utility providers (electricity, water, gas, broadband).',
   'User can enter the bill account number to fetch the bill amount.',
   'User can make instant payments using any linked bank account.',
   'System provides a payment confirmation and receipt post-transaction.']},
 {'story_id': 'US-005',
  'title': 'Purchase Insurance Through the App',
  'description': 'As a user, I want to purchase insurance through the app, so that I can protect myself and my family from unforeseen events.',
  'acceptance_criteria': ['User can browse and select different insurance products (e.g., life, health, auto).',
   'User can view insurance details including coverage, premium, and terms.',
   'User can purchase insurance directly through the app with required documentation.',
   'System provides confirmation and policy documents post-purchase.']}]


In [96]:
project_title = "PayMate: Your Ultimate Payment Companion"
project_description = "PayMate is a comprehensive payment application that allows users to perform seamless transactions using the Unified Payments Interface (UPI). Beyond basic payments, PayMate offers features such as quick loans, bill payments, and a user-friendly interface, making it a one-stop solution for all financial needs."
requirements = ["Implement multi-factor authentication, including biometrics (fingerprint and facial recognition) and MPIN, to secure user accounts.​", "Enable users to link multiple bank accounts and perform instant fund transfers using UPI.", "Provide users with access to instant micro-loans with minimal documentation.", "Allow users to pay utility bills such as electricity, water, gas, and broadband directly through the app."]
requirements_string = "\n".join(requirements)

project_requirements = f"Project Title: {project_title}\nProject Description: {project_description}\nRequirements: {requirements_string}"

In [97]:
def generate_frontend_code_from_llm(project_requirements):
    user_query =  f"Create a frontend react + vite code for the project requirements: {project_requirements}."
    chain = prompt | llm 
    response = chain.invoke({"system_prompt" : CODE_SYSTEM_PROMPT, "human_query" : user_query})
    return response.content

def revised_frontend_code_from_llm(code, user_feedback):
    frontend_code = "console.log('Revised frontend code')"
    return frontend_code

In [98]:
frontend_code = generate_frontend_code_from_llm(user_stories)

In [99]:
frontend_code

'Let\'s set up a React + Vite project for PayMate with the required user stories.\n\n<boltArtifact id="paymate-frontend" title="PayMate Frontend Setup">\n<boltAction type="file" filePath="package.json">\n{\n  "name": "paymate",\n  "private": true,\n  "version": "0.0.1",\n  "type": "module",\n  "scripts": {\n    "dev": "vite",\n    "build": "vite build",\n    "preview": "vite preview"\n  },\n  "dependencies": {\n    "@emotion/react": "^11.11.1",\n    "@emotion/styled": "^11.11.0",\n    "@mui/icons-material": "^5.14.9",\n    "@mui/material": "^5.14.9",\n    "react": "^18.2.0",\n    "react-dom": "^18.2.0",\n    "react-router-dom": "^6.15.0",\n    "zustand": "^4.4.1"\n  },\n  "devDependencies": {\n    "@types/react": "^18.2.15",\n    "@types/react-dom": "^18.2.7",\n    "@vitejs/plugin-react": "^4.0.3",\n    "vite": "^4.4.5"\n  }\n}\n</boltAction>\n\n<boltAction type="file" filePath="src/App.jsx">\nimport { BrowserRouter, Routes, Route } from \'react-router-dom\'\nimport Layout from \'./com

In [ ]:
'Let\'s set up a React + Vite project for PayMate with the required user stories.\n\n<boltArtifact id="paymate-frontend" title="PayMate Frontend Setup">\n<boltAction type="file" filePath="package.json">\n{\n  "name": "paymate",\n  "private": true,\n  "version": "0.0.1",\n  "type": "module",\n  "scripts": {\n    "dev": "vite",\n    "build": "vite build",\n    "preview": "vite preview"\n  },\n  "dependencies": {\n    "@emotion/react": "^11.11.1",\n    "@emotion/styled": "^11.11.0",\n    "@mui/icons-material": "^5.14.9",\n    "@mui/material": "^5.14.9",\n    "react": "^18.2.0",\n    "react-dom": "^18.2.0",\n    "react-router-dom": "^6.15.0",\n    "zustand": "^4.4.1"\n  },\n  "devDependencies": {\n    "@types/react": "^18.2.15",\n    "@types/react-dom": "^18.2.7",\n    "@vitejs/plugin-react": "^4.0.3",\n    "vite": "^4.4.5"\n  }\n}\n</boltAction>\n\n<boltAction type="file" filePath="src/App.jsx">\nimport { BrowserRouter, Routes, Route } from \'react-router-dom\'\nimport Layout from \'./components/Layout\'\nimport Home from \'./pages/Home\'\nimport Authentication from \'./pages/Authentication\'\nimport BankAccounts from \'./pages/BankAccounts\'\nimport Loans from \'./pages/Loans\'\nimport Bills from \'./pages/Bills\'\nimport Insurance from \'./pages/Insurance\'\n\nexport default function App() {\n  return (\n    <BrowserRouter>\n      <Routes>\n        <Route path="/" element={<Layout />}>\n          <Route index element={<Home />} />\n          <Route path="auth" element={<Authentication />} />\n          <Route path="accounts" element={<BankAccounts />} />\n          <Route path="loans" element={<Loans />} />\n          <Route path="bills" element={<Bills />} />\n          <Route path="insurance" element={<Insurance />} />\n        </Route>\n      </Routes>\n    </BrowserRouter>\n  )\n}\n</boltAction>\n\n<boltAction type="file" filePath="src/main.jsx">\nimport React from \'react\'\nimport ReactDOM from \'react-dom/client\'\nimport { ThemeProvider, CssBaseline } from \'@mui/material\'\nimport theme from \'./theme\'\nimport App from \'./App\'\n\nReactDOM.createRoot(document.getElementById(\'root\')).render(\n  <React.StrictMode>\n    <ThemeProvider theme={theme}>\n      <CssBaseline />\n      <App />\n    </ThemeProvider>\n  </React.StrictMode>\n)\n</boltAction>\n\n<boltAction type="file" filePath="src/theme.js">\nimport { createTheme } from \'@mui/material\'\n\nexport default createTheme({\n  palette: {\n    primary: {\n      main: \'#1976d2\'\n    },\n    secondary: {\n      main: \'#dc004e\'\n    }\n  }\n})\n</boltAction>\n\n<boltAction type="file" filePath="src/components/Layout.jsx">\nimport { Outlet } from \'react-router-dom\'\nimport { Box, Container } from \'@mui/material\'\nimport Navbar from \'./Navbar\'\n\nexport default function Layout() {\n  return (\n    <Box>\n      <Navbar />\n      <Container component="main" sx={{ mt: 4 }}>\n        <Outlet />\n      </Container>\n    </Box>\n  )\n}\n</boltAction>\n\n<boltAction type="file" filePath="src/components/Navbar.jsx">\nimport { AppBar, Toolbar, Typography, Button } from \'@mui/material\'\nimport { Link } from \'react-router-dom\'\n\nexport default function Navbar() {\n  return (\n    <AppBar position="static">\n      <Toolbar>\n        <Typography variant="h6" component={Link} to="/" sx={{ flexGrow: 1, textDecoration: \'none\', color: \'inherit\' }}>\n          PayMate\n        </Typography>\n        <Button color="inherit" component={Link} to="/auth">Auth</Button>\n        <Button color="inherit" component={Link} to="/accounts">Accounts</Button>\n        <Button color="inherit" component={Link} to="/loans">Loans</Button>\n        <Button color="inherit" component={Link} to="/bills">Bills</Button>\n        <Button color="inherit" component={Link} to="/insurance">Insurance</Button>\n      </Toolbar>\n    </AppBar>\n  )\n}\n</boltAction>\n\n<boltAction type="file" filePath="index.html">\n<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="UTF-8" />\n    <meta name="viewport" content="width=device-width, initial-scale=1.0" />\n    <title>PayMate</title>\n  </head>\n  <body>\n    <div id="root"></div>\n    <script type="module" src="/src/main.jsx"></script>\n  </body>\n</html>\n</boltAction>\n\n<boltAction type="shell">\nnpm install && npm run dev\n</boltAction>\n</boltArtifact>'

## Backend code

In [100]:
def generate_backend_code_from_llm(project_requirements, user_stories):
    backend_code = "print('backend code')"
    return backend_code 

def revised_backend_code_from_llm(code, user_feedback):
    backend_code = "print('Revised backend code')"
    return backend_code